In [1]:
import pandas as pd
import os

In [3]:
start_date = '2014-03-01'
end_date = '2025-03-10'
path = os.path.join('..', 'data', f'nvidia_data_{start_date}_{end_date}')

In [ ]:
class Preprocessor(object):
    def __init__(self):
        